In [2]:
import os
os.chdir('../')
%pwd

'd:\\Programming\\aiTeaching'

In [25]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: Path
    local_data_file: Path
    unzip_dir: Path
    contests_dir: Path
    authors_file: Path 

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    data_file: Path
    train_file: Path
    test_file: Path
    tokenizer_name: Path

In [4]:
from src.aiteacher.constants import *
from src.aiteacher.utils.common import read_yaml, create_directories

In [26]:
class ConfigurationManager:
    def __init__(self, config_path=CONFIG_FILE_PATH,
                 params_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_path)
        self.params=read_yaml(params_path)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
            contests_dir=config.contests_dir,
            authors_file=config.authors_file
        )
        return data_ingestion_config
    
    def get_data_transformation_config(self)-> DataTransformationConfig:
        config=self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            data_file=config.data_file,
            train_file=config.train_file,
            test_file=config.test_file,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config

In [6]:
import os
import urllib.request as request
from src.aiteacher import logger
import subprocess
import pandas as pd
import bz2

In [10]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.unzip_dir):
            create_directories([self.config.unzip_dir])
            logger.info(f"Cloning dataset from {self.config.source_url}")
            subprocess.run(["git", "clone", self.config.source_url, self.config.unzip_dir], check=True)
            logger.info("Dataset sucessfully cloned.")
        else:
            logger.info(f"Dataset already exists at {self.config.unzip_dir}")

    def decompress_bz2(self, file_path, output_path):
        """Decompress a .bz2 file to a CSV file."""
        with bz2.BZ2File(file_path, "rb") as f_in:
            with open(output_path, "wb") as f_out:
                f_out.write(f_in.read())

    def extract_data(self):
        try:
            contests_path = self.config.contests_dir
            file_paths = [f for f in os.listdir(contests_path) if f.endswith(".csv.bz2")][:5]

            all_contests = []
            temp_dir = os.path.join(contests_path, "temp_extracted")
            os.makedirs(temp_dir, exist_ok=True)

            for file in file_paths:
                contest_id = file.split(".")[0]
                compressed_file_path = os.path.join(contests_path, file)
                extracted_file_path = os.path.join(temp_dir, f"{contest_id}.csv")
                
                self.decompress_bz2(compressed_file_path, extracted_file_path)
                
                df = pd.read_csv(extracted_file_path, engine="python", on_bad_lines="skip")
                df["contest_id"] = contest_id
                all_contests.append(df)
                logger.info(f"Loaded {len(df)} records from contest {contest_id}")

            if all_contests:
                save_path = self.config.local_data_file
                create_directories([os.path.dirname(save_path)])
                for i, df in enumerate(all_contests): 
                    file_name = f"contest_{i+1}.csv"  
                    file_path = os.path.join(save_path, file_name)
                    df.to_csv(file_path, index=False)  
                logger.info(f"All contests saved to {save_path}.")
                return save_path
        except Exception as e:
            logger.error(f"Error extracting data: {e}")
            raise

In [11]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(config=data_ingestion_config)
data_ingestion.download_file()
data_ingestion.extract_data()

[2025-03-16 02:40:24,798: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-16 02:40:24,801: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-16 02:40:24,801: INFO: common: created directoiy at: artifacts]
[2025-03-16 02:40:24,802: INFO: common: created directoiy at: artifacts/data_ingestion]
[2025-03-16 02:40:24,802: INFO: 1350481930: Dataset already exists at artifacts/data_ingestion/raw]
[2025-03-16 02:40:24,823: INFO: 1350481930: Loaded 107 records from contest 1]
[2025-03-16 02:40:24,903: INFO: 1350481930: Loaded 939 records from contest 10]
[2025-03-16 02:40:25,020: INFO: 1350481930: Loaded 891 records from contest 100]
[2025-03-16 02:40:25,410: INFO: 1350481930: Loaded 6139 records from contest 1000]
[2025-03-16 02:40:25,629: INFO: 1350481930: Loaded 2602 records from contest 1001]
[2025-03-16 02:40:25,630: INFO: common: created directoiy at: artifacts/data_ingestion/data]
[2025-03-16 02:40:25,827: INFO: 1350481930: All contests saved to

'artifacts/data_ingestion/data/'

In [19]:
df = pd.read_csv("artifacts/data_ingestion/data/contest_1.csv")
df.columns

Index(['Unnamed: 0', 'contest_id', 'submission_id', 'author', 'author_url',
       'problem', 'problem_url', 'language', 'verdict', 'time', 'memory',
       'sent', 'judged', 'source_code'],
      dtype='object')

In [14]:
from src.aiteacher.constants import *
from src.aiteacher.utils.common import read_yaml,create_directories

In [32]:
import os
from src.aiteacher import logger
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from datasets import load_from_disk

In [34]:
class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config=config
        self.tokenizer=AutoTokenizer.from_pretrained(config.tokenizer_name)

    def data_transformation(self, test_size=0.2, random_state=42):
        if not os.path.exists(self.config.train_file):
            config = self.config
            data_path = config.data_path
            all_dfs = []
            for file in os.listdir(data_path):
                file_path = os.path.join(data_path, file)
                df = pd.read_csv(file_path, usecols=["language", "verdict", "source_code"])
                all_dfs.append(df)
            if not all_dfs:
                logger.error(f"No csv files found at {data_path}")
                return
            
            data_file = config.data_file
            train_file = config.train_file
            test_file = config.test_file
            merged_df = pd.concat(all_dfs, ignore_index=True)
            merged_df.to_csv(data_file, index=False)
            train_df, test_df = train_test_split(merged_df, test_size=test_size, random_state=random_state)

            train_df.to_csv(train_file, index=False)
            test_df.to_csv(test_file, index=False)

            logger.info(f"Data split successfully! Train: {len(train_df)}, Test: {len(test_df)}")
        else:
            logger.info(f"Data Transformation has already occurred")


    def convert_examples_to_features(self, example_batch):
        input_text = example_batch['source_code']

        input_encodings = self.tokenizer(
            input_text,
            max_length=1024,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        return {
            'input_ids': input_encodings['input_ids'].squeeze(0),
            'attention_mask': input_encodings['attention_mask'].squeeze(0),
            'language': example_batch['language'],
            'verdict': example_batch['verdict']
        }
    
    def convert(self):
        train_dataset = load_from_disk(self.config.train_file)
        train_dataset_pt = train_dataset.map(self.convert_examples_to_features, batched = True)
        train_dataset_pt.save_to_disk(os.path.join(self.config.root_dir,"train_dataset"))
        logger.info(f"Successfully tokenized train dataset")


In [ ]:
config = ConfigurationManager()
data_transformation_config = config.get_data_transformation_config()
data_transformation = DataTransformation(config=data_transformation_config)
data_transformation.data_transformation()
data_transformation.convert()

[2025-03-16 03:12:32,216: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-16 03:12:32,218: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-16 03:12:32,220: INFO: common: created directoiy at: artifacts]
[2025-03-16 03:12:32,222: INFO: common: created directoiy at: artifacts/data_transformation]
[2025-03-16 03:12:33,530: INFO: 641851836: Data split successfully! Train: 25627, Test: 6407]
